In [1]:
%reload_ext lab_black
from pprint import pprint
from worstcase import config, param, derive, unit

In [6]:
config.n = 2000  # Number of Monte Carlo runs. (default: 5000)
config.sigfig = 3  # Number of significant figures to print. (default: 4)

In [7]:
VIN = param.byrange(nom=5 * unit.V, lb=4.8 * unit.V, ub=5.1 * unit.V, tag="voltage-in")
RTOP = param.bytol(nom=5 * unit.kohm, tol=0.05, rel=True, tag="top-resistor")
RBOT = param.bytol(nom=15 * unit.kohm, tol=0.10, rel=True, tag="bottom-resistor")

pprint([VIN, RTOP, RBOT])

[voltage-in: 5 V (nom), 4.8 V (lb), 5.1 V (ub),
 top-resistor: 5 kΩ (nom), 4.75 kΩ (lb), 5.25 kΩ (ub),
 bottom-resistor: 15 kΩ (nom), 13.5 kΩ (lb), 16.5 kΩ (ub)]


In [8]:
@derive.bymc(vin=VIN, rtop=RTOP, rbot=RBOT, tag="voltage-out")
def vout(vin, rtop, rbot):
    return vin * rbot / (rtop + rbot)


VOUT = vout()

print(VOUT)

voltage-out: 3.75 V (nom), 3.47 V (lb), 3.94 V (ub)


In [18]:
vout(4.8, 5.25, 13.5)

3.456

In [2]:
import worstcase as wca
import numpy as np
from pprint import pprint

wca.config.n = 2000  # Number of Monte Carlo runs. (default: 5000)
wca.config.sigfig = 4  # Number of significant firues to print. (default: 3)

In [3]:
spd_initial = wca.param.bytol(nom=2, tol=0.1, rel=True, unit=wca.unit("m/s"), tag="v0")
accel = wca.param.byrange(nom=0.2, lb=0.1, ub=0.5, unit=wca.unit("m/s**2"), tag="a")
distance = wca.param.byrange(nom=1, lb=0.8, ub=1.1, unit=wca.unit.km, tag="x")

pprint([spd_initial, accel, distance])

[v0: 2 m/s (nom), 1.8 m/s (lb), 2.2 m/s (ub),
 a: 200 mm/s² (nom), 100 mm/s² (lb), 500 mm/s² (ub),
 x: 1 km (nom), 800 m (lb), 1.1 km (ub)]


In [4]:
@wca.param.ev(spd_initial, accel, distance)
def spd_final(v, a, x):
    return np.sqrt(v ** 2 + 2 * a * x)


print(spd_final)

spd_final (ev)
├── a
├── v0
└── x



In [5]:
print(spd_final())

spd_final: 20.1 m/s (nom), 12.78 m/s (lb), 33.24 m/s (ub)


In [6]:
spd_final_noaccel = spd_final(a=0 * wca.unit("m/s**2"), tag="spd_noaccel")
print(spd_final_noaccel)

spd_noaccel (ev)
├── v0
└── x



In [7]:
result = spd_final_noaccel(3 * wca.unit("m/s"), x=10 * wca.unit.m)
print(result)

3.0 meter / second


In [8]:
spd_rel = wca.param.bytol(nom=20, tol=1, rel=False, unit=wca.unit("mi/hr"), tag="vrel")


@wca.param.mc(spd_final, spd_rel)
def spd_total(vf, vr):
    return vf + vr


print(spd_total)
print(spd_total())

spd_total (mc)
├── spd_final (ev)
│   ├── a
│   ├── v0
│   └── x
└── vrel

spd_total: 29.04 m/s (nom), 21.41 m/s (lb), 42.57 m/s (ub)


In [9]:
accel_sens = spd_total.ss(accel, tag="accel-sens")
print(accel_sens)
print(accel_sens())

accel-sens (mc)
└── spd_final (ev)
    └── a

accel-sens: 29.04 m/s (nom), 23.23 m/s (lb), 40.62 m/s (ub)


In [10]:
accel_distance_sens = spd_total.ss([accel, distance], tag="accel/distance-sens")
print(accel_distance_sens)
print(accel_distance_sens())

accel/distance-sens (mc)
└── spd_final (ev)
    ├── a
    └── x

accel/distance-sens: 29.04 m/s (nom), 21.75 m/s (lb), 42.16 m/s (ub)


In [11]:
finalspd_sens = spd_total.ss(spd_final, tag="finalspd-sens")
print(finalspd_sens)
print(finalspd_sens())

finalspd-sens (mc)
└── spd_final (ev)
    ├── a
    ├── v0
    └── x

finalspd-sens: 29.04 m/s (nom), 21.72 m/s (lb), 42.17 m/s (ub)


In [12]:
relspd_sens = spd_total.ss(spd_rel, tag="relspd-sens")
print(relspd_sens)
print(relspd_sens())

relspd-sens (mc)
└── vrel

relspd-sens: 29.04 m/s (nom), 28.59 m/s (lb), 29.49 m/s (ub)


In [13]:
assert spd_total.check("[length]/[time]")
# also try: spd_total.dimensionality

In [14]:
print(spd_total.units)
# also try: spd_total.u

meter / second


In [15]:
print(spd_total().ito(wca.unit("km/hr")))
# also try: spd_total().ito_base_units()
#           spd_total().ito_reduced_units()
#           spd_total().ito_root_units()

spd_total: 104.5 km/hr (nom), 76.93 km/hr (lb), 153.1 km/hr (ub)
